In [131]:
import boto3
import json

In [132]:
bd_clt = boto3.client("bedrock-runtime", region_name="us-east-1")

In [134]:
prompt = """
You are an AI SQL assistant. Generate SQL queries based on natural language prompts. Use the given schema as reference.
Generate sql statement inside a tag <SQL>. Also include finetune_llm_querygen database name with a dot in outputquery

Schema:
{
  "tables": {
    "employee": {
      "columns": {
        "employee_id": "int",
        "first_name": "varchar",
        "last_name": "varchar",
        "department_id": "int",
        "hire_date": "date"
      }
    },
    "department": {
      "columns": {
        "department_id": "int",
        "department_name": "varchar"
      }
    },
    "services": {
      "columns": {
        "service_id": "int",
        "service_name": "varchar"
      }
    },
    "enrollment": {
      "columns": {
        "employee_id": "int",
        "service_id": "int",
        "enrollment_date": "date"
      }
    }
  }
}

Examples:
User: List all employees in the IT department.
AI: SELECT * FROM "employee" e JOIN department d ON e.department_id = d.department_id WHERE d.department_name = 'IT';

User: Get all employees hired after January 1, 2021.
AI: SELECT * FROM "employee" WHERE hire_date > "2021-01-01";

User: Get all services available in the system.
AI: SELECT * FROM "services";

User: Find all employees with the last name 'Smith'.
AI: SELECT * FROM "employee" WHERE last_name = 'Smith';

User: Get the department details of employees hired before 2020.
AI: SELECT e.first_name, e.last_name, d.department_name FROM "employee" e JOIN department d ON e.department_id = d.department_id WHERE e.hire_date < '2020-01-01';

User: List all services employees have enrolled in.
AI: SELECT s.service_name, e.first_name, e.last_name FROM "employee" en JOIN services s ON en.service_id = s.service_id JOIN employee e ON en.employee_id = e.employee_id;

User: Which employees are enrolled in the Health Insurance service?
AI: SELECT e.first_name, e.last_name FROM "employee" en JOIN services s ON en.service_id = s.service_id JOIN employee e ON en.employee_id = e.employee_id WHERE s.service_name = 'Health Insurance';

User: What is the total number of employees in each department?
AI: SELECT d.department_name, COUNT(e.employee_id) AS employee_count FROM "employee" e JOIN department d ON e.department_id = d.department_id GROUP BY d.department_name;

User: Find employees who have never enrolled in a service.
AI: SELECT e.first_name, e.last_name FROM "employee" e LEFT JOIN enrollment en ON e.employee_id = en.employee_id WHERE en.employee_id IS NULL;

User: Which employees are enrolled in multiple services?
AI: SELECT e.first_name, e.last_name, COUNT(en.service_id) AS service_count FROM "enrollment" en JOIN employee e ON en.employee_id = e.employee_id GROUP BY e.first_name, e.last_name HAVING COUNT(en.service_id) > 1;

User: What is the total number of enrollments per service?
AI: SELECT s.service_name, COUNT(en.service_id) AS total_enrollments FROM "enrollment" en JOIN services s ON en.service_id = s.service_id GROUP BY s.service_name;

Now, generate the SQL query for the following request:
User: Find all employees with the last name 'Anderson'?
AI: 
"""

In [135]:
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
bedrock_runtime = boto3.client('bedrock-runtime')
user_message = {"role": "user", "content": prompt}
messages = [user_message]
max_tokens = 1024
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "system": prompt,
        "messages": messages
    }
)
response = bedrock_runtime.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())

In [136]:
print(response_body)

{'id': 'msg_bdrk_01DCknbnx4EkRDW9oKdzRr71', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': "Here's the SQL query to find all employees with the last name 'Anderson':\n\n<SQL>\nSELECT * FROM finetune_llm_querygen.employee WHERE last_name = 'Anderson';\n</SQL>\n\nThis query will return all columns for employees whose last name is 'Anderson' from the employee table in the finetune_llm_querygen database."}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1848, 'output_tokens': 86}}


In [122]:
sql_stmt = response_body['content'][0]['text']
print(sql_stmt)

Here's the SQL query to find all employees with the last name 'Anderson':

<SQL>
SELECT * FROM finetune_llm_querygen.employee WHERE last_name = 'Anderson';
</SQL>

This query selects all columns for employees whose last name is 'Anderson' from the employee table in the finetune_llm_querygen database.


In [123]:
import re
pattern = r'<SQL>(.*?)</SQL>'
match = re.search(pattern, sql_stmt, re.DOTALL)
print(match)

<re.Match object; span=(75, 162), match="<SQL>\nSELECT * FROM finetune_llm_querygen.employ>


In [124]:
sql_content = match.group(1)
print(sql_content)


SELECT * FROM finetune_llm_querygen.employee WHERE last_name = 'Anderson';



In [125]:
athena_client = boto3.client("athena", region_name = 'us-east-1')

bucket_name = "query-gen-hackathon"

In [126]:
response = athena_client.start_query_execution(QueryString= sql_content, QueryExecutionContext = {'Database': 'finetune_llm_querygen', 'Catalog': 'AwsDataCatalog'},
                                              ResultConfiguration = {'OutputLocation': "s3://query-gen-hackathon/athena-query-result", 
                                                                    })
query_id = response['QueryExecutionId']
print(query_id)

7b50c0a5-22dd-4be9-aaca-074ac86c80d0


In [127]:
exec_status = athena_client.get_query_execution(QueryExecutionId=query_id)
print(exec_status)

{'QueryExecution': {'QueryExecutionId': '7b50c0a5-22dd-4be9-aaca-074ac86c80d0', 'Query': "SELECT * FROM finetune_llm_querygen.employee WHERE last_name = 'Anderson'", 'StatementType': 'DML', 'ResultConfiguration': {'OutputLocation': 's3://query-gen-hackathon/athena-query-result/7b50c0a5-22dd-4be9-aaca-074ac86c80d0.csv'}, 'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}}, 'QueryExecutionContext': {'Database': 'finetune_llm_querygen', 'Catalog': 'awsdatacatalog'}, 'Status': {'State': 'RUNNING', 'SubmissionDateTime': datetime.datetime(2025, 3, 3, 15, 58, 51, 877000, tzinfo=tzlocal())}, 'Statistics': {'TotalExecutionTimeInMillis': 351, 'QueryQueueTimeInMillis': 63, 'ServicePreProcessingTimeInMillis': 193, 'ResultReuseInformation': {'ReusedPreviousResult': False}}, 'WorkGroup': 'primary', 'EngineVersion': {'SelectedEngineVersion': 'AUTO', 'EffectiveEngineVersion': 'Athena engine version 3'}, 'SubstatementType': 'SELECT'}, 'ResponseMetadata': {'RequestId': 'a21

In [137]:
execution_id = exec_status.get('QueryExecution').get('QueryExecutionId')
print(execution_id)
import time
time.sleep(3)

7b50c0a5-22dd-4be9-aaca-074ac86c80d0


In [138]:
s3_client = boto3.client('s3')
resp = s3_client.get_object(Bucket = bucket_name, Key = f'athena-query-result/{execution_id}.csv')
print(resp['Body'])

In [139]:
import pandas as pd
output = pd.read_csv(resp['Body'])
print("****")
print(output)

****
   employee_id first_name last_name  department_id   hire_date
0            4     Angela  Anderson            102  2021-02-19
1           15     Brandi  Anderson            102  2023-12-13
2           67    Garrett  Anderson            102  2023-08-09
3           94        Jon  Anderson            102  2021-10-07
